# Full Adder
Below is a Logisim diagram a Full Adder circuit implementation. 
![](images/full_adder_logisim.png)

In [1]:
from magma import *
from magma.backend.verilog import compile as compile_verilog

![](images/SB_LUT4_diagram.png)
![](images/SB_LUT4_verilog.png)

In [2]:
SB_LUT4 = DeclareCircuit('SB_LUT4',
                         "I0", In(Bit),
                         "I1", In(Bit),
                         "I2", In(Bit),
                         "I3", In(Bit),
                         "O",  Out(Bit))

A0 = 0xAAAA
A1 = 0xCCCC
A2 = 0xF0F0
A3 = 0xFF00

In [3]:
class And2(Circuit):
    name = "And2"
    IO = ["I0", In(Bit), "I1", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0&A1, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(0, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)

class Or3(Circuit):
    name = "Or3"
    IO = ["I0", In(Bit), "I1", In(Bit), "I2", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0|A1|A3, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(io.I2, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)


class Xor3(Circuit):
    name = "Xor3"
    IO = ["I0", In(Bit), "I1", In(Bit), "I2", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0^A1^A3, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(io.I2, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)

In [4]:
class FullAdder(Circuit):
    name = "FullAdder"
    IO = ["a", In(Bit), "b", In(Bit), "cin", In(Bit), 
          "out", Out(Bit), "cout", Out(Bit)]
    @classmethod
    def definition(io):
        # Generate the sum
        out = Xor3()(io.a, io.b, io.cin)
        wire(out, io.out)
        # Generate the carry
        a_and_b = And2()(io.a, io.b)
        b_and_cin = And2()(io.b, io.cin)
        a_and_cin = And2()(io.a, io.cin)
        cout = Or3()(a_and_b, b_and_cin, a_and_cin)
        wire(cout, io.cout)


In [5]:
print(compile_verilog(FullAdder))

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
module Xor3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9966)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input  I0, input  I1, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I0), .I1(I1), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hFFEE)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module FullAdder (input  a, input  b, input  cin, output  out, output  cout);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
Xor3 inst0 (.I0(a), .I1(b), .I2(cin), .O(inst0_O));
And2 inst1 (.I0(a), .I1(b), .O(inst1_O));
And2 inst2 (.I0(b), .I1(cin), .O(inst2_O));
And2 inst3 (.I0(a), .I1(cin), .O(inst3_O));
Or3 inst4 (.I0(inst1_O), .I1(

In [6]:
def DefineAdder(width):
    T = UInt(width)
    class Adder(Circuit):
        name = "Adder{}".format(width)
        IO = ["a", In(T), "b", In(T), "cin", In(Bit), "out", Out(T), "cout", Out(Bit)]
        @classmethod
        def definition(io):
            adders = [FullAdder() for _ in range(width)]
            circ = braid(adders, foldargs={"cin":"cout"})
            wire(io.a, circ.a)
            wire(io.b, circ.b)
            wire(io.cin, circ.cin)
            wire(io.cout, circ.cout)
            wire(io.out, circ.out)
    return Adder

In [7]:
print(compile_verilog(DefineAdder(4)))

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
compiling Adder4
module Xor3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9966)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input  I0, input  I1, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I0), .I1(I1), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hFFEE)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module FullAdder (input  a, input  b, input  cin, output  out, output  cout);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
Xor3 inst0 (.I0(a), .I1(b), .I2(cin), .O(inst0_O));
And2 inst1 (.I0(a), .I1(b), .O(inst1_O));
And2 inst2 (.I0(b), .I1(cin), .O(inst2_O));
And2 inst3 (.I0(a), .I1(cin), .O(inst3_O));
Or3 inst4 (.

# Register
Below is the documentation for the `SB_DFFE` module for the Lattice iCE architecture.
![](images/SB_DFFE_diagram.png)

In magma we can declare a circuit that implements the `SB_DFFE` interface using `DeclareCircuit`

In [8]:
SB_DFFE = DeclareCircuit('SB_DFFE',
    "C", In(Bit), "D", In(Bit), "E", In(Bit), "Q", Out(Bit))

In [9]:
def DefineRegister(width):
    T = UInt(width)
    class Register(Circuit):
        name = "Register{}".format(width)
        IO = ["I", In(T), "O", Out(T), "CLK", In(Bit), "CE", In(Bit)]
        @classmethod
        def definition(io):
            ffs = join([SB_DFFE() for _ in range(width)])
            wire(io.I, ffs.D)
            wire(io.O, ffs.Q)
            wire([io.CLK for _ in range(width)], ffs.C)
            wire([io.CE for _ in range(width)], ffs.E)
    return Register

In [10]:
print(compile_verilog(DefineRegister(4)))

compiling Register4
module Register4 (input [3:0] I, output [3:0] O, input  CLK, input  CE);
wire  inst0_Q;
wire  inst1_Q;
wire  inst2_Q;
wire  inst3_Q;
SB_DFFE inst0 (.C(CLK), .D(I[0]), .E(CE), .Q(inst0_Q));
SB_DFFE inst1 (.C(CLK), .D(I[1]), .E(CE), .Q(inst1_Q));
SB_DFFE inst2 (.C(CLK), .D(I[2]), .E(CE), .Q(inst2_Q));
SB_DFFE inst3 (.C(CLK), .D(I[3]), .E(CE), .Q(inst3_Q));
assign O = {inst3_Q,inst2_Q,inst1_Q,inst0_Q};
endmodule




In [11]:
def DefineCounter(width):
    T = UInt(width)
    class Counter(Circuit):
        name = "Counter{}".format(width)
        IO = ["O", Out(T), "cout", Out(Bit), "CLK", In(Bit), "CE", In(Bit)]
        @classmethod
        def definition(io):
            reg = DefineRegister(width)()
            adder = DefineAdder(width)()
            wire(reg.O, adder.a)
            wire(int2seq(0, width), adder.b)
            wire(1, adder.cin)
            wire(adder.out, reg.I)
            wire(adder.cout, io.cout)
            wire(reg.O, io.O)
            wire(io.CLK, reg.CLK)
            wire(io.CE, reg.CE)
    return Counter

In [12]:
print(compile_verilog(DefineCounter(4)))

compiling Register4
compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
compiling Adder4
compiling Counter4
module Register4 (input [3:0] I, output [3:0] O, input  CLK, input  CE);
wire  inst0_Q;
wire  inst1_Q;
wire  inst2_Q;
wire  inst3_Q;
SB_DFFE inst0 (.C(CLK), .D(I[0]), .E(CE), .Q(inst0_Q));
SB_DFFE inst1 (.C(CLK), .D(I[1]), .E(CE), .Q(inst1_Q));
SB_DFFE inst2 (.C(CLK), .D(I[2]), .E(CE), .Q(inst2_Q));
SB_DFFE inst3 (.C(CLK), .D(I[3]), .E(CE), .Q(inst3_Q));
assign O = {inst3_Q,inst2_Q,inst1_Q,inst0_Q};
endmodule

module Xor3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9966)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input  I0, input  I1, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I0), .I1(I1), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_

In [13]:
from loam.boards.icestick import IceStick
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()

main = icestick.main()
counter4 = DefineCounter(4)()
counter23 = DefineCounter(23)()
wire(1, counter23.CE)
wire(counter23.cout, counter4.CE)

wire(counter4.O, [main.D1, main.D2, main.D3, main.D4])

# print(compile_verilog(main))

import mantle lattice ice40
import mantle lattice mantle40


In [14]:
compile("build/ice_counter", main)

compiling Register4
compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
compiling Adder4
compiling Counter4
compiling Register23
compiling Adder23
compiling Counter23
compiling main


In [15]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_counter.blif' ice_counter.v
arachne-pnr -q -d 1k -o ice_counter.txt -p ice_counter.pcf ice_counter.blif
icepack ice_counter.txt ice_counter.bin

# sudo kextunload -b com.apple.driver.AppleUSBFTDI
iceprog ice_counter.bin
# sudo kextload -b com.apple.driver.AppleUSBFTDI

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
